In [1]:
! pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.4 MB 9.6 MB/s eta 0:00:01
   --------------- ------------------------ 3.7/9.4 MB 10.9 MB/s eta 0:00:01
   -------------------------- ------------- 6.3/9.4 MB 11.0 MB/s eta 0:00:01
   ------------------------------------- -- 8.7/9.4 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 9.9 MB/s eta 0:00:00


In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Caminho do ChromeDriver no seu sistema
caminho_driver = r"C:\Users\user\Desktop\chromedriver\chromedriver.exe"  # <- ajuste se colocou em outra pasta

# Configuração do navegador
chrome_options = Options()
chrome_options.add_argument("--headless")  # opcional: roda sem abrir janela

# Cria o navegador
service = Service(caminho_driver)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Acessa a página
url = "https://www.meutimao.com.br/artilharia-do-corinthians/todos-os-tempos/"
driver.get(url)

#Tentando descobrir pq o HTML não carrega
#As vezes é necessário esperar um pouco para a página carregar
try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "relatorio"))
    )
    print("ul.relatorio carregada!")
except:
    print("ul.relatorio NÃO foi carregada...")

#Descobrindo que fui bloqueado
html = driver.page_source
with open("pagina.html", "w", encoding="utf-8") as f:
    f.write(html)
print("HTML salvo.")


# Pega o HTML renderizado
soup = BeautifulSoup(driver.page_source, 'html.parser')

ul.relatorio NÃO foi carregada...
HTML salvo.


In [12]:
jogadores = []

# Encontra a <ul class="relatorio">
ul_relatorio = soup.find('ul', class_='relatorio')

# Garante que a ul foi encontrada
if ul_relatorio:
    for jogador in ul_relatorio.find_all('li'):
        try:
            nome = jogador.select_one('h5 a span')
            info = jogador.select_one('div.info p em')

            # Verifica se os elementos foram encontrados antes de acessar o texto
            nome_texto = nome.get_text(strip=True) if nome else "Nome não encontrado"
            info_texto = info.get_text(strip=True) if info else "Info não encontrada"

            jogadores.append([nome_texto, info_texto])

        except Exception as e:
            print(f"Erro ao processar jogador: {e}")
else:
    print("ul.relatorio não encontrada!")

# Exibe os jogadores coletados
print(jogadores)

driver.quit()

ul.relatorio não encontrada!
[]


In [9]:
import pandas as pd
df = pd.DataFrame(jogadores, columns=['Nome', 'Gols'])
df.to_csv('jogadores.csv', index=False)